In [1]:
from pyomo.environ import *

# Define sets and indices
P = {'GCA', 'GCB', 'GCC'}  # Products
C = {'C1', 'C2'}  # Components

# Define parameters (using your sample data)
a = {'C1': 10000, 'C2': 15000}  # Available liters of each component
d = {'GCA': 6000, 'GCB': 7000, 'GCC': 9000}  # Minimum demand for each product
e = {'C1': 0.4, 'C2': 0.2}  # Critical element proportion in each component
l = {'GCA': 0.3, 'GCB': 0.3}  # Minimum required proportion of critical element
r_GCC = 0.3  # Minimum ratio of C1 to C2 in GCC
p = {'GCA': 120, 'GCB': 135, 'GCC': 155}  # Profit per liter of each product

# Create the model
model = ConcreteModel()

# Define decision variables
model.x = Var(C, P, domain=NonNegativeReals)

total_profit_expr = sum(p[j] * sum(model.x[i, j] for i in C) for j in P)  #### HUMAN ADDED

# Define objective function (maximize profit)
model.profit = Objective(expr=total_profit_expr, sense=maximize)

# Define constraints

# 1. Component availability
def component_availability_rule(model, i):
    return sum(model.x[i, j] for j in P) <= a[i]

model.component_availability = Constraint(C, rule=component_availability_rule)

# 2. Minimum demand
def minimum_demand_rule(model, j):
    return sum(model.x[i, j] for i in C) >= d[j]

model.minimum_demand = Constraint(P, rule=minimum_demand_rule)

crit_element_gca_expr = sum(e[i] * model.x[i, 'GCA'] for i in C) >= l['GCA'] * sum(model.x[i, 'GCA'] for i in C) #### HUMAN ADDED

# 3. Critical element requirement for GCA
model.critical_element_GCA = Constraint(expr = crit_element_gca_expr)

crit_element_gcb_expr = sum(e[i] * model.x[i, 'GCB'] for i in C) <= l['GCB'] * sum(model.x[i, 'GCB'] for i in C) #### HUMAN ADDED

# 4. Critical element requirement for GCB
model.critical_element_GCB = Constraint(expr = crit_element_gcb_expr)

c1c2_ratio_expr = model.x['C1', 'GCC'] >= r_GCC * model.x['C2', 'GCC']
# 5. C1/C2 ratio for GCC
model.ratio_GCC = Constraint(expr=c1c2_ratio_expr)

# Solve the model
solver = SolverFactory('glpk')  # You can use other solvers like 'cbc', 'gurobi', etc.
solver.solve(model)

# Print the results

print("Optimal Solution:")
for j in P:
    print(f"Product {j}:")
    for i in C:
        print(f"  Component {i}: {model.x[i, j]():.2f} liters")
print(f"Total Profit: {model.profit():.2f}")



Optimal Solution:
Product GCC:
  Component C1: 2769.23 liters
  Component C2: 9230.77 liters
Product GCB:
  Component C1: 3500.00 liters
  Component C2: 3500.00 liters
Product GCA:
  Component C1: 3730.77 liters
  Component C2: 2269.23 liters
Total Profit: 3525000.00
